In [2]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
from collections import deque

# YOLO modeli ile önceden eğitilmiş şişe tespit modeli yükleniyor
yolo_model = YOLO("D:/Bitirme Projesi - İnsan Hareketlerini Sınıflandıran Yapay Zeka Sistemi/YOLOv11s ile Eğitilen Model Çıktıları/weights/best.pt")

# MediaPipe eller modülü başlatılıyor
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# --- Sistem parametreleri ---
min_points_in_bbox = 5  # Bir bounding box içine düşmesi gereken minimum el noktası sayısı
movement_threshold = 2  # Nesne merkezinin hareket ettiğini anlamak için gereken minimum piksel farkı
motion_stable_frames = 5  # Sabit kalma için gereken ardışık kare sayısı
message_stable_threshold = 5  # Durum mesajının sabitlenmesi için gereken tekrar sayısı
rotation_queue_size = 10  # Parmak hareket yönü için kullanılacak kuyruk boyutu
angle_threshold = 15  # Şişe aç/kapa jesti için minimum çapraz vektör açısı (derece)
shake_movement_threshold = 30  # Sallanma algısı için el hareket eşik değeri
shake_frame_threshold = 3  # Sallanma için gerekli ardışık frame sayısı

# --- Pouring (dökme) parametreleri ---
INDEX_FINGER = 8  # İşaret parmağı MediaPipe index
PINKY_FINGER = 20  # Serçe parmak MediaPipe index
MAX_VERTICAL_DEVIATION_PERCENT = 5  # Dikey hizalanma toleransı (% olarak ekran yüksekliğine göre)
POURING_FRAMES_THRESHOLD = 10  # Dökme pozisyonunun sabit kalması gereken süre (frame sayısı)
POURING_ANGLE_THRESHOLD = 30  # Dökme için gereken minimum eğim açısı (derece cinsinden)
pouring_frame_count = 0
pouring_state = False  # Su dökülme durumu

# Açma/kapama mesajı sabitleme parametreleri
OPEN_CLOSE_DISPLAY_FRAMES = 30  # Açma/kapama mesajının ekranda kalma süresi (frame)
open_close_display_counter = 0
open_close_display_message = ""

# Hareket durum değişkenleri
object_state = "Yok"  # Başlangıçta elde nesne yok
tracked_object = None  # Takip edilen bounding box
prev_bbox_center = None  # Önceki bounding box merkezi
flags = {"holding": False, "moving": False, "dropped": False, "stopped": False}  # Elindeki nesne durumu
stable_frame_count = 0  # Hareketsiz kalan frame sayısı
message_counter = {"Yok": 0, "Aldi": 0, "Tasiniyor": 0, "Birakti": 0, "Aldi (durdu)": 0}  # Mesaj sabitleyici
current_display_message = ""

# Şişe açma/kapama jest yönü takibi
rotation_queue = deque(maxlen=rotation_queue_size)
open_close_state = ""
rotation_frame_counter = 0

# Sallama hareketi takibi
shake_positions = deque(maxlen=5)
shake_frame_count = 0
shake_state = False

# Kamera kaynağını başlat
cap = cv2.VideoCapture(0)

# Noktanın bounding box içinde olup olmadığını kontrol eder
def is_point_in_bbox(point, bbox):
    x, y = point
    x_min, y_min, x_max, y_max = bbox
    return x_min <= x <= x_max and y_min <= y <= y_max

# Bounding box merkezini döndürür
def get_bbox_center(bbox):
    x_min, y_min, x_max, y_max = bbox
    return ((x_min + x_max) // 2, (y_min + y_max) // 2)

# İki bounding box arasındaki IoU (kesişim/union oranı) hesaplar
def calculate_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter_area = max(0, xB - xA) * max(0, yB - yA)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

# El parmak hizasının dikey farkını kontrol eder (pouring için)
def check_finger_alignment(hand_landmarks, frame_height):
    if hand_landmarks is None:
        return False
    index_tip = hand_landmarks.landmark[INDEX_FINGER]
    pinky_tip = hand_landmarks.landmark[PINKY_FINGER]
    index_y = int(index_tip.y * frame_height)
    pinky_y = int(pinky_tip.y * frame_height)
    max_deviation = int(frame_height * MAX_VERTICAL_DEVIATION_PERCENT / 100)
    vertical_diff = abs(index_y - pinky_y)
    return vertical_diff <= max_deviation

# Bounding box üzerinden şişenin eğim açısını hesaplar (dökme hareketi için)
def calculate_bbox_angle(bbox):
    x_min, y_min, x_max, y_max = bbox
    width = x_max - x_min
    height = y_max - y_min
    if width == 0:
        return 0
    angle = np.degrees(np.arctan2(height, width))
    return angle

# Ana döngü - her video karesi için çalışır
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO ile nesne tespiti
    results = yolo_model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

    # MediaPipe ile el landmark tespiti
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(frame_rgb)

    hand_points = []
    fingertip_points = []
    first_hand_landmarks = None

    # Tüm ellerin parmak uçlarını ve landmarklarını kaydet
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            lm = hand_landmarks.landmark
            for i in [8, 12, 16, 20]:
                x, y = int(lm[i].x * frame.shape[1]), int(lm[i].y * frame.shape[0])
                fingertip_points.append((x, y))
            hand_points.extend([(int(l.x * frame.shape[1]), int(l.y * frame.shape[0])) for l in lm])
            if idx == 0:
                first_hand_landmarks = hand_landmarks

    # Yalnızca şişe sınıfına ait bounding box'ları filtrele
    bottle_boxes = [boxes[i] for i, cid in enumerate(class_ids) if cid == 0]

    # Şişe bulunduysa kontrol yap
    if len(bottle_boxes) > 0:
        for bbox in bottle_boxes:
            x_min, y_min, x_max, y_max = bbox.astype(int)
            points_in_bbox = [pt for pt in hand_points if is_point_in_bbox(pt, (x_min, y_min, x_max, y_max))]

            if len(points_in_bbox) >= min_points_in_bbox:
                # El şişe üzerindeyse, durumu güncelle
                if object_state in ["Birakti", "Aldi (Durdu)"]:
                    object_state = "Aldi"
                    flags.update({"holding": True, "moving": False, "dropped": False, "stopped": False})

                current_bbox_center = get_bbox_center((x_min, y_min, x_max, y_max))

                if prev_bbox_center is not None:
                    distance = np.linalg.norm(np.array(current_bbox_center) - np.array(prev_bbox_center))
                    if distance > movement_threshold:
                        flags.update({"moving": True, "holding": True, "dropped": False, "stopped": False})
                        stable_frame_count = 0
                        object_state = "Tasiniyor"
                    else:
                        stable_frame_count += 1
                        if stable_frame_count >= motion_stable_frames:
                            flags.update({"holding": True, "moving": False, "dropped": False, "stopped": False})
                            object_state = "Aldi"
                else:
                    flags.update({"holding": True, "moving": False, "dropped": False})
                    object_state = "Aldi"

                tracked_object = (x_min, y_min, x_max, y_max)
                prev_bbox_center = current_bbox_center
            else:
                # El artık şişede değilse: bırakıldı say
                if flags["holding"] or flags["moving"]:
                    flags.update({"dropped": True, "holding": False, "moving": False, "stopped": False})
                    object_state = "Birakti"
                tracked_object = None
                prev_bbox_center = None

        # Şişe var ama hiç el yoksa da: bırakıldı say
        if not hand_points and (flags["holding"] or flags["moving"] or flags["stopped"]):
            flags.update({"dropped": True, "holding": False, "moving": False, "stopped": False})
            object_state = "Birakti"
    else:
        # Şişe yoksa tüm durumları sıfırla
        object_state = "Yok"
        tracked_object = None
        prev_bbox_center = None
        flags = {"holding": False, "moving": False, "dropped": False, "stopped": False}

    # Mesaj sabitleme mekanizması
    for key in message_counter:
        message_counter[key] = message_counter[key] + 1 if key == object_state else 0
    if message_counter[object_state] >= message_stable_threshold + 1:
        current_display_message = object_state

    # Açma/kapama yönü tespiti için parmak hareketi analizi
    if len(fingertip_points) >= 3:
        rotation_queue.append(np.mean(fingertip_points, axis=0))
        if len(rotation_queue) >= 3:
            vec1 = rotation_queue[-3] - rotation_queue[-2]
            vec2 = rotation_queue[-1] - rotation_queue[-2]
            cross = np.cross(vec1, vec2)
            if np.abs(cross) > angle_threshold:
                rotation_frame_counter += 1
                open_close_state = "Kapatiliyor" if cross < 0 else "Aciliyor"
            else:
                rotation_frame_counter = 0
                open_close_state = ""
        else:
            open_close_state = ""
    else:
        open_close_state = ""

    # Açma/kapama mesajını ekranda gösterme süresi
    if open_close_state and flags["holding"]:
        open_close_display_message = open_close_state
        open_close_display_counter = OPEN_CLOSE_DISPLAY_FRAMES
    elif open_close_display_counter > 0:
        open_close_display_counter -= 1
        if open_close_display_counter == 0:
            open_close_display_message = ""

    # Sallanma tespiti: el merkezinin hareket analizine göre
    if len(hand_points) > 5:
        current_hand_center = np.mean(hand_points, axis=0)
        shake_positions.append(current_hand_center)
        if len(shake_positions) >= 2:
            movement = np.linalg.norm(shake_positions[-1] - shake_positions[-2])
            if movement > shake_movement_threshold:
                shake_frame_count += 1
            else:
                shake_frame_count = max(0, shake_frame_count - 1)
        else:
            shake_frame_count = 0
        shake_state = shake_frame_count >= shake_frame_threshold
    else:
        shake_state = False
        shake_frame_count = 0
        shake_positions.clear()

    # --- Pouring (dökme) hareketi tespiti ---
    # Şişe elde tutuluyorsa ve el hizalanması + şişe eğimi uygunsa dökme say
    if first_hand_landmarks is not None and flags["holding"] and tracked_object is not None:
        is_aligned = check_finger_alignment(first_hand_landmarks, frame.shape[0])
        angle = calculate_bbox_angle(tracked_object)
        if is_aligned and angle < POURING_ANGLE_THRESHOLD:
            pouring_frame_count += 1
            if pouring_frame_count >= POURING_FRAMES_THRESHOLD:
                pouring_state = True
        else:
            pouring_frame_count = max(0, pouring_frame_count - 1)
            if pouring_frame_count == 0:
                pouring_state = False
    else:
        pouring_state = False
        pouring_frame_count = 0

    # Görsel çizimler ve durum yazıları
    for i, bbox in enumerate(boxes):
        x1, y1, x2, y2 = bbox.astype(int)
        color = (0, 255, 0) if class_ids[i] == 0 else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # Ekrana durum yazıları ekle
    cv2.putText(frame, f"Durum: {current_display_message}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)

    if open_close_display_message and not shake_state and not pouring_state:
        cv2.putText(frame, f"Sise {open_close_display_message}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 100, 0), 2)

    if shake_state and not pouring_state:
        cv2.putText(frame, "Sise sallaniyor", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 140, 255), 2)

    if pouring_state:
        cv2.putText(frame, "SU DOKULUYOR!", (frame.shape[1]//2-150, frame.shape[0]//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    # Ekranı göster
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kaynakları serbest bırak
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 14.9ms
Speed: 1.5ms preprocess, 14.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.9ms
Speed: 1.5ms preprocess, 14.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.8ms
Speed: 1.8ms preprocess, 13.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.0ms
Speed: 1.4ms preprocess, 14.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.8ms
Speed: 1.2ms preprocess, 16.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 16.0ms
Speed: 1.6ms preprocess, 16.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 34.8ms
Speed: 4.4ms preprocess, 34.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.7ms
Speed: 1.3ms preprocess, 13.7ms i